# Tenor and Alto saxophones

* Sample duration = 5s
* model = SVM
* target = TENOR and ALTO

Distinguish between samples with tenor saxophone or alto saxophone from those without tenor saxophone or alto saxophone. Other brass and (most) woodwind instruments excluded.

In [3]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, \
    RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import uniform

In [4]:
df = pd.read_pickle('./data/5s/labeled/features_r02/all_data.pkl')

In [5]:
NUM_LABEL_COLS = 13
NUM_COMPONENTS = 26
RANDOM_SEED = 0

In [6]:
df_filtered = df
df_filtered.shape

(9080, 10790)

In [7]:
# exclude records we want to exclude
df_filtered = df_filtered[df_filtered['sop'] == '0']
df_filtered = df_filtered[df_filtered['bari'] == '0']
df_filtered = df_filtered[df_filtered['clrt'] == '0']
df_filtered = df_filtered[df_filtered['othr'] == '0']
df_filtered = df_filtered[df_filtered['trmp'] == '0']
df_filtered = df_filtered[df_filtered['trmb'] == '0']
df_filtered = df_filtered[df_filtered['otrb'] == '0']

num_x_cols = df_filtered.shape[1] - NUM_LABEL_COLS - 1  
# ^ last bit to adjust for zero indexing
data = df_filtered.iloc[:, 1:num_x_cols].to_numpy() 
# ^ These are the features
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

# Create target column
combined = df_filtered[['tenr']].to_numpy() + df_filtered[['alto']].to_numpy() + df_filtered[['tora']].to_numpy()
combined = combined.astype('int')
combined[combined > 0] = 1
df_filtered['tenor/alto'] = combined

target = df_filtered[['tenor/alto']].to_numpy().ravel()  # << This is the label

print(data.shape)
print(target.shape)

(7368, 10775)
(7368,)


In [8]:
pca = PCA(n_components=NUM_COMPONENTS)

In [9]:
pca.fit(data)

PCA(n_components=26)

In [10]:
d = pca.transform(data)

In [11]:
x_train, x_test, y_train, y_test = \
     train_test_split(d, target, test_size=0.20, random_state=0)

## Test plain vanilla SVM

In [11]:
clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8704206241519674


In [12]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,482,97
1,94,801


In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.83      0.83       579
           1       0.89      0.89      0.89       895

    accuracy                           0.87      1474
   macro avg       0.86      0.86      0.86      1474
weighted avg       0.87      0.87      0.87      1474



## Determine good value for C using random search

In [14]:
model_params = {
    'C': uniform(0.1, 10.0)
}

# RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, 
#                    scoring=None, n_jobs=None, iid='deprecated', 
#                    refit=True, cv=None, verbose=0, 
#                    pre_dispatch='2*n_jobs', random_state=None, 
#                    error_score=nan, return_train_score=False)
svm_model = SVC()
clf = RandomizedSearchCV(svm_model, model_params, n_iter=10, 
                         cv=5)
model = clf.fit(x_train, y_train)
params = model.best_estimator_.get_params()

In [20]:
params


{'C': 8.879914721794334,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [15]:
clf = SVC(C=8.879914721794334)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))


0.8792401628222524


In [16]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,499,80
1,98,797


In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85       579
           1       0.91      0.89      0.90       895

    accuracy                           0.88      1474
   macro avg       0.87      0.88      0.87      1474
weighted avg       0.88      0.88      0.88      1474



## Test ensemble approach with bagging classifier

In [18]:
# Parameters
p = {'max_features': [NUM_COMPONENTS - 2, NUM_COMPONENTS - 3, 
                      NUM_COMPONENTS - 4, NUM_COMPONENTS - 5,
                      NUM_COMPONENTS - 6, NUM_COMPONENTS - 7]}

base_estimator = SVC(C=8.879914721794334)
estimator = BaggingClassifier(base_estimator=base_estimator, n_estimators=100)
# Question: Should we include C here, or ... ? There's no reason it should differ
# for different values of n_estimators, but what about max_features? Might different
# values for C be best for different max_features? Maybe. Dunno. Maybe we should
# search for optimal C *after* we've found the best value for max_features?

# GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, 
#              iid='deprecated', refit=True, cv=None, verbose=0, 
#              pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
clf = GridSearchCV(estimator, p, n_jobs=-1)
search = clf.fit(x_train, y_train)
search.best_params_

# {'max_features': 19}

{'max_features': 19}

In [ ]:
# BaggingClassifier(base_estimator=None, n_estimators=10, *, 
#                   max_samples=1.0, max_features=1.0, 
#                   bootstrap=True, bootstrap_features=False, 
#                   oob_score=False, warm_start=False, 
#                   n_jobs=None, random_state=None, verbose=0) 
clf = BaggingClassifier(base_estimator=SVC(C=8.879914721794334,
                                           probability=True),
                        n_estimators=500, 
                        max_features=19,
                        random_state=RANDOM_SEED,
                        n_jobs=-1)
model = clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

In [ ]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
fn = './scikit_models/alto_+_tenor_svm_tuned_c.pkl'
with open(fn, 'wb') as fh:
    pickle.dump(clf, fh)